In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langchain.agents import AgentExecutor, Tool
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,AIMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_structured_chat_agent

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model="gpt-4o",temperature=0.1)

In [4]:
def get_local_time(input: str = None) -> str:
    from datetime import datetime
    current_time = datetime.now().strftime("%H:%M:%S")
    # print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

In [5]:
def create_stickies_note(note_text):
    import subprocess
    
    # erase special symbols
    note_text = note_text.replace('"', '\\"')
    # allow start a new line
    note_text = note_text.replace("\n", '" & return & "')
    
    applescript = f'''
    tell application "Stickies"
        activate
    end tell
    delay 1
    tell application "System Events"
        keystroke "n" using {{command down}} -- create new stick
        delay 1
        keystroke "{note_text}" -- note input
    end tell
    '''
    subprocess.run(["osascript", "-e", applescript])

    return f"Agentfinished, you just need to response the note is done."

In [6]:
tools = [
    Tool(
        name="LocalTime",
        func=get_local_time,
        description="Use this tool to get the current local time."
    ),
    Tool(
        name="CreateNote",
        func=create_stickies_note,
        description="Use this tool to write down a note"
    )
]

In [7]:
system_message=SystemMessagePromptTemplate.from_template(
    """ Respond to the human as helpfully and accurately as possible. You have access to the following tools:
        {tools}
        Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
        Valid "action" values: "Final Answer" or {tool_names}.
        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}
        ```
        ** And if you invoked any tool, return the tool's return content with your own words. Do not more iterations.**
                
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
     
        """
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [8]:
prompt_template=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message,
    assisstant_message
])

In [9]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_8007/1976451665.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [10]:
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt_template)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory,handle_parsing_errors=True, verbose=True)

In [12]:
response = agent_executor.invoke({
    "input": "What is 15^2?",
    "history": memory.chat_memory.messages,  
    # "agent_scratchpad": ''
})

print(response)



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Final Answer",
  "action_input": "The result of 15^2 is 225."
}
```

> Finished chain.
{'input': 'What is 15^2?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='The result of 15^2 is 225.', additional_kwargs={}, response_metadata={})], 'output': 'The result of 15^2 is 225.'}


In [13]:
response['output']

'The result of 15^2 is 225.'

In [14]:
response = agent_executor.invoke({
    "input": "What is the time now?",
    "history": memory.chat_memory.messages,  
})

print(response)



> Entering new AgentExecutor chain...
Thought: I need to use the LocalTime tool to get the current local time.
Action:
```json
{
  "action": "LocalTime",
  "action_input": ""
}
```Current local time is: 16:51:38Action:
```json
{
  "action": "Final Answer",
  "action_input": "The current local time is 16:51:38."
}
```

> Finished chain.
{'input': 'What is the time now?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='The result of 15^2 is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 16:51:38.', additional_kwargs={}, response_metadata={})], 'output': 'The current local time is 16:51:38.'}


In [15]:
response['output']

'The current local time is 16:51:38.'

In [16]:
response = agent_executor.invoke({
    "input": "I plan to go out for a workout today at about 9 am in the moring and then I have a online catchup, write down as a note.",
    "history": memory.chat_memory.messages,  
})

print(response['output'])



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "CreateNote",
  "action_input": "Plan for today: Go out for a workout at about 9 AM in the morning, followed by an online catchup."
}
```Agentfinished, you just need to response the note is done.Action:
```json
{
  "action": "Final Answer",
  "action_input": "The note has been created: Plan for today: Go out for a workout at about 9 AM in the morning, followed by an online catchup."
}
```

> Finished chain.
The note has been created: Plan for today: Go out for a workout at about 9 AM in the morning, followed by an online catchup.


In [17]:
from langchain_core.prompts import PromptTemplate

In [18]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, you should: 
think about the input content and rewrite the content in your own language based on the content and theme, making sure to keep the wording and format concise. 
** Format note with bullet point, each point as a row**.
# When calling a tool, always use the following format:

# ```
# {{
#     "action": "ConcatenateStrings",
#     "action_input": {{
#     "a": "<first_string>",
#     "b": "<second_string>",
#     ...
#     }}
# }}
# ```
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

In [19]:
prompt_template=PromptTemplate.from_template(template)

In [20]:
tools = [
    Tool(
        name="CreateNote",
        func=create_stickies_note,
        description="Use this tool to write down a note"
    )
]

In [21]:
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)

In [22]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template,
    stop_sequence=True,
)

In [23]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [25]:
response = agent_executor.invoke({"input": "Artificial Intelligence (AI) is a rapidly evolving field that enables machines to mimic human intelligence, including learning, problem-solving, and decision-making. From voice assistants and recommendation systems to self-driving cars and medical diagnostics, AI is transforming industries and daily life. Powered by algorithms and vast amounts of data, AI continuously improves through machine learning and deep learning techniques. While it offers immense benefits, such as automation and efficiency, AI also raises ethical concerns regarding privacy, bias, and job displacement. As AI continues to advance, responsible development and regulation will be crucial to ensuring its positive impact on society."})



> Entering new AgentExecutor chain...
To address the question, I need to create a concise note summarizing the key points about AI, its applications, benefits, and challenges. 

Action: CreateNote
Action Input: 
- AI mimics human intelligence: learning, problem-solving, decision-making.
- Applications: voice assistants, recommendation systems, self-driving cars, medical diagnostics.
- Transforms industries and daily life.
- Powered by algorithms and data.
- Improves through machine learning and deep learning.
- Benefits: automation, efficiency.
- Challenges: ethical concerns about privacy, bias, job displacement.
- Importance of responsible development and regulation for positive societal impact.
Agentfinished, you just need to response the note is done.I now know the final answer.

Final Answer: The note summarizing the key points about AI has been created successfully.

> Finished chain.
